# Calculate normalised enrichment scores of Bindea gene sets

Script related to figure 3b

### Aim:
- Calculate TCGA OV NES of Bindea Gene Sets

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load expression data

In [ ]:
mix <- "../../Data/1/TCGAov_RNAseq.txt"

In [3]:
data <- read.csv(mix,
                 sep='\t',
                 header=T,
                 row.names='GeneSymbol')

In [4]:
head(data)

,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,TCGA.09.0367.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
A1BG,66.4695,65.5664,41.6412,187.0368,23.9295,32.8123,46.3418,162.8522,88.5518,192.7354,⋯,63.7012,99.1485,38.3684,140.3668,94.7103,70.4394,43.1736,33.6854,27.9604,16.2092
A1BG-AS1,36.3243,28.1315,23.2465,114.6008,10.4957,20.1900,17.2007,83.0906,65.6758,55.1265,⋯,30.3602,71.1797,26.0239,77.2415,61.4890,106.1623,47.8864,25.3482,17.9853,42.9450
A1CF,0.0000,0.0000,0.3310,0.0000,0.0000,0.0000,0.0000,0.9034,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,1.9057,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A2M,5899.8279,9384.4401,3350.4207,1455.2316,3999.3792,3224.5797,1986.1593,1487.3247,10557.8325,4603.9013,⋯,10264.1373,7220.7664,7200.1144,1819.9428,6179.7358,5365.7972,8773.5936,10006.7877,1888.1746,4205.6055
A2M-AS1,118.4566,111.0026,71.3613,67.8607,52.1501,224.7582,182.0650,120.2838,53.6419,92.9112,⋯,146.6130,45.8648,64.4638,28.5565,100.6605,23.5625,84.2153,582.3492,59.8084,143.4910
A2ML1,7.5289,54.6875,5.6263,5.0883,3.3148,3.7421,6.2305,278.2486,4.7763,653.6058,⋯,1.7993,4.6124,0.4768,17.1510,15.6109,2.4461,98.3162,269.5768,4.5092,224.5290


In [5]:
data_mtx <- as.matrix(data)

## Load Bindea gene sets

In [6]:
Bindea_GeneSets <- read.csv('../1.Data/BindeaImmuneCells_GeneSets.txt',
                            sep='\t',
                            header=T)

In [7]:
head(Bindea_GeneSets)

activated_Dendritic_cells,B_cells,CD8_T_cells,Citotoxic_cells,Dendritic_cells,Eosinophils,immature_Dendritic_cells,Macrophages,Mast_cells,Neutrophils,⋯,T_central_memory_cells,T_effector_memory_cells,T_follicular_helper_cells,T_gamma_delta_cells,Th1_cells,Th17_cells,Th2_cells,Treg_cells,Angiogenesis,Antigen_presentation_machinery
CCL1,ABCB4,ABT1,APBA2,CCL13,ABHD2,ABCG2,APOE,ABCC4,ALPL,⋯,AQP3,AKT3,B3GAT1,C1orf61,APBB2,IL17A,ADCY1,FOXP3,CDH5,HLA-A
EBI3,BACH2,AES,APOL3,CCL17,ACACB,BLVRB,ATG7,ADCYAP1,BST1,⋯,ATF7IP,C7orf54,BLR1,CD160,APOD,IL17RA,AHI1,,ELTD1,HLA-B
INDO,BCL11A,APBA2,CTSW,CCL22,C9orf156,CARD9,BCAT1,CALB2,CD93,⋯,ATM,CCR2,C18orf1,FEZ1,ATP9A,RORC,ANK1,,CLEC14A,HLA-C
LAMP3,BLK,ARHGAP8,DUSP2,CD209,CAT,CD1A,CCL7,CEACAM8,CEACAM3,⋯,CASP8,DDX17,CDK5R1,TARP,BST2,,BIRC5,,LDB2,B2M
OAS3,BLNK,C12orf47,GNLY,HSD11B1,CCR3,CD1B,CD163,CMA1,CREB5,⋯,CDC14A,EWSR1,CHGB,TRD,BTG3,,CDC25C,,ECSCR,TAP1
,CCR9,C19orf6,GZMA,NPR1,CLC,CD1C,CD68,CPA3,CRISPLD2,⋯,CEP68,FLI1,CHI3L2,TRGV9,CCL4,,CDC7,,MYCT1,TAP2


In [8]:
Bindea_GeneSets <- convertRowsToList(t(Bindea_GeneSets))

## Run ssGSEA

In [9]:
bin_NES <- gsva(data_mtx,
                Bindea_GeneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 26 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [10]:
bin_NES <- data.frame("Term"=rownames(bin_NES),
                      bin_NES)

rownames(bin_NES) <- NULL

In [11]:
head(bin_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
activated_Dendritic_cells,0.12080193,0.10556223,0.13333816,-0.05315550,0.190243596,0.07906845,0.03492371,-0.10237760,0.10161003,⋯,-0.01959475,0.18322489,0.10781454,-0.065679466,0.02578157,0.168298851,0.07877285,-0.16003707,0.05411122,0.02013184
B_cells,-0.03962222,0.02903597,-0.09843530,-0.07195296,-0.091762622,-0.10770259,-0.08408320,-0.09445385,-0.07808819,⋯,-0.13033419,-0.02076153,-0.03244857,-0.029069976,-0.04435688,-0.086043574,-0.07494591,-0.07293641,-0.09804945,-0.08942158
CD8_T_cells,0.22723342,0.26320113,0.20884786,0.19328751,0.209855174,0.19964907,0.19570424,0.19654905,0.18947288,⋯,0.18528949,0.19154989,0.19474076,0.171608853,0.21566246,0.182533036,0.14478878,0.16828649,0.18368081,0.21155103
Citotoxic_cells,0.03149694,0.07878722,-0.07434844,-0.16349677,-0.005047341,-0.17442586,-0.15453201,-0.18036433,-0.02391936,⋯,-0.12771299,-0.06213032,-0.04333806,-0.103947834,-0.06120171,-0.032273609,-0.19110453,-0.18296366,-0.09313858,-0.14620225
Dendritic_cells,-0.01598181,0.03919308,-0.02865189,-0.14344404,-0.034157532,-0.18025025,-0.03126540,-0.09952214,-0.08081835,⋯,-0.05318072,0.02456869,-0.03191424,-0.009081463,-0.05925390,-0.004000022,-0.06908451,-0.04662226,-0.09573182,-0.13796367
Eosinophils,0.08082758,0.10824983,0.11646350,0.08816655,0.085352121,0.13576262,0.08916077,0.11674802,0.12588133,⋯,0.13083037,0.10553173,0.09535649,0.103924202,0.13292447,0.094407569,0.07948510,0.07270317,0.07892876,0.08980974


### Save NES

In [12]:
write.table(bin_NES,
            file='../1.Data/BindeaImmuneCells_TCGAov_NES.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script